# 501 Prepare data for global assessment

In this notebook, we prepare the data necessary for our global assessment.

In [1]:
import pyam
import pandas as pd
import numpy as np
import scmdata
from pathlib import Path
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/scmdata/database/_database.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook as tqdman


Step 1: Read in the necessary data.

In [2]:
cdr_original = pyam.IamDataFrame(
    Path(
        '../data/100_novel_cdr.csv'
    )
)

pyam.core - INFO: Reading file ../data/100_novel_cdr.csv


In [3]:
cdr_climate = pyam.IamDataFrame(
    Path(
        '../data/405_cdr_170324.csv'
    )
)

pyam.core - INFO: Reading file ../data/405_cdr_170324.csv


In [4]:
metrics = pd.read_csv(
    Path('../data/401_lookup.csv'),
    index_col=[0,1]
)

Step 2: Now, per ensemble member, we want to calculate the additional cumulative CDR between net zero CO2 and 2100.

In [5]:
additional_cdr_climate = []

In [6]:
cdr_original.interpolate(
    time=range(2015, 2101),
    inplace=True
)

In [7]:
for (model, scenario) in tqdm(cdr_original.meta.index):
    # Filter for original CDR
    cdr_orig = (
        cdr_original
        .filter(
            model=model,
            scenario=scenario
        )
        .timeseries()
    )
    # Filter for adapted CDR
    cdr_alt = (
        cdr_climate
        .filter(
            model=model,
            scenario=scenario,
        )
        .timeseries()
    )
    # Subtract original from adapted
    cdr_diff = pyam.IamDataFrame(cdr_alt - cdr_orig)
    additional_cdr_climate.append(
        pyam.IamDataFrame(cdr_diff)
    )

  0%|          | 0/95 [00:00<?, ?it/s]

In [8]:
additional_cdr_climate = pyam.concat(additional_cdr_climate)

In [9]:
additional_cdr_climate.rename(
    variable={
        'Carbon Dioxide Removal|Novel':'Carbon Dioxide Removal|Novel|Additional'
    },
    inplace=True
)

Step 3: Convert all the necessary units to Gt CO2

In [10]:
additional_cdr_climate.convert_unit(
    current='Mt CO2/yr',
    to='Gt CO2/yr',
    inplace=True
)

In [11]:
cdr_original.convert_unit(
    current='Mt CO2/yr',
    to='Gt CO2/yr',
    inplace=True
)

Step 4: Now, we want to calculate the cumulative additional CDR from net zero CO2 to 2100. We can just sum up across all values for each timeseries entry, since we have already calculated the difference from the original model trajectory.

In [12]:
cumulative_cdr_additional = (
    additional_cdr_climate
    .timeseries()
    .apply(
        lambda x: format(pyam.timeseries.cumulative(
            x,
            first_year=2020,
            last_year=2100
        ),'f'),
        axis=1
    )
)

Step 5: Assign this as an additional column to the metrics dataframe. Then write this out for further analysis.

In [13]:
metrics = metrics.set_index('run_id', append=True)

In [14]:
for (mod, scen, run) in metrics.index:
    if mod == 'GEM-E3_V2021':
        continue
    metrics.loc[(mod, scen, run), 'additional_cdr_gtco2_final'] = (
        cumulative_cdr_additional
        .loc[pd.IndexSlice[mod, scen, :, :, :, :, run]]
        .values[0]
    )

In [15]:
metrics['additional_cdr_gtco2_final'] = metrics['additional_cdr_gtco2_final'].astype('float').round(2)

In [16]:
metrics.to_excel(
    Path(
        '../data/501_metrics_with_final_cdr.xlsx'
    ),
    merge_cells=False
)